In [2]:
#pip install pycryptodome
#pip install sympy

In [16]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Util import number
from sympy.ntheory.factor_ import totient
import binascii
import struct
import random

prime_base = 8
unicode_base = 256
enc_msg_base = pow(2,prime_base*2) 
# This can be either generated by existing library (1) or calculated manually from the formulas (2)

"""
# Option 1
keyPair = RSA.generate(1024)
 
pubKey = keyPair.publickey()
print(f"Public key:  (n={hex(pubKey.n)}, e={hex(pubKey.e)})")
pubKeyPEM = pubKey.exportKey()
print(pubKeyPEM.decode('ascii'))
 
print(f"Private key: (n={hex(pubKey.n)}, d={hex(keyPair.d)})")
privKeyPEM = keyPair.exportKey()
print(privKeyPEM.decode('ascii'))
"""

#Option 2
p = number.getPrime(prime_base)
q = number.getPrime(prime_base)
n = p*q
n_tot = totient(n)
e = random.randint(3, 101)
while n_tot % e == 0:
    e = random.randint(3, 101)
    
# Modular multiplication inverse formula to calculate d, taken from internet.
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m
d = modinv(e, n_tot)


print("P:",p)
print("Q:",q)
print("N:",n)
print("E:",e)
print("D:",d)
print("Totient(N):",n_tot)

P: 179
Q: 251
N: 44929
E: 47
D: 19883
Totient(N): 44500


In [40]:
# Encryption for Strings

# Step 1 - Converting the input into binary/ascii/hex
msg_string = "A message for encryption"
msg_ascii = [ord(c) for c in msg_string]
msg_binary = [f'{c:08b}' for c in msg_ascii]
msg_hex = [hex(int(c,2)) for c in msg_binary]

# Step 2 - Hex to hex string
msg_hex_string = "0x"
for c in msg_hex:
    msg_hex_string += c[2:4]
print(msg_hex_string)

# Step 3 - Taking the power of input so that enc_msg = (msg ^ e) % n
enc_msg = []
for c in msg_ascii:
    temp = pow(c,e) % n
    enc_msg.append(temp)
    
# Step 4 - Converting the encrypted input into binary/hex
enc_msg_binary_original = [f'{c:016b}' for c in enc_msg]
enc_msg_binary_v8  = []
enc_msg_hex = []
for c in enc_msg_binary:
    for i in range(int(len(c)/8)):
        temp2 = c[i*8:i*8+8]
        enc_msg_binary_v8.append(temp2)
        enc_msg_hex.append(hex(int(temp2,2)))

# Step 5 - Encrypted hex to encrypted hex string
        
enc_msg_hex_string = "0x"
for c in enc_msg_hex:
    enc_msg_hex_string += c[2:4]
print(enc_msg_hex_string)

0x41206d65737361676520666f7220656e6372797074696f6e
0x6d4746843130a8926e856e85e1d649ca89246841bfe9e3d79774684a89282de44e0797785ba99437a7a71a9e3d82de


In [28]:
# Decryption for Strings

# Step 1 - Converting the hex string into decryptable segments
for 
dec_msg = []
for c in enc_msg:
    temp = pow(c,d) % n
    dec_msg.append(temp)
dec_msg_binary_original = [f'{c:08b}' for c in dec_msg]
dec_msg_hex = []
for c in dec_msg_binary:
    for i in range(int(len(c)/8)):
        temp2 = c[i*8:i*8+8]
        dec_msg_binary_original.append(temp2)
        dec_msg_hex.append(hex(int(temp2,2)))
enc_msg_hex_string = "0x"
for c in enc_msg_hex:
    enc_msg_hex_string += c[2:4]
print(enc_msg_hex_string)

n


In [41]:
print(msg_string)
print(msg_ascii)
print(msg_binary)
print(msg_hex)
print(msg_hex_string)
print(enc_msg)
print(enc_msg_binary_original)
print(enc_msg_binary_v8)
print(enc_msg_hex)
print(enc_msg_string)

A message for encryption
[65, 32, 109, 101, 115, 115, 97, 103, 101, 32, 102, 111, 114, 32, 101, 110, 99, 114, 121, 112, 116, 105, 111, 110]
['01000001', '00100000', '01101101', '01100101', '01110011', '01110011', '01100001', '01100111', '01100101', '00100000', '01100110', '01101111', '01110010', '00100000', '01100101', '01101110', '01100011', '01110010', '01111001', '01110000', '01110100', '01101001', '01101111', '01101110']
['0x41', '0x20', '0x6d', '0x65', '0x73', '0x73', '0x61', '0x67', '0x65', '0x20', '0x66', '0x6f', '0x72', '0x20', '0x65', '0x6e', '0x63', '0x72', '0x79', '0x70', '0x74', '0x69', '0x6f', '0x6e']
0x41206d65737361676520666f7220656e6372797074696f6e
[27975, 18052, 12592, 43154, 28293, 28293, 3613, 25756, 43154, 18052, 7166, 40509, 31095, 18052, 43154, 33502, 17632, 31095, 34059, 43412, 14247, 42778, 40509, 33502]
['0110110101000111', '0100011010000100', '0011000100110000', '1010100010010010', '0110111010000101', '0110111010000101', '0000111000011101', '0110010010011100',